In [1]:
import numpy as np
import pandas as pd
import pymysql
import configparser
from decimal import Decimal
import json

import warnings
warnings.filterwarnings("ignore")

import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from datetime import datetime

In [2]:
# 连接数据库
def getConn():
    config = configparser.ConfigParser()
    config.read('config.ini', encoding='utf8')
    db_url = config['data_center_config']['host']
    db_port = config['data_center_config']['port']
    db_username = config['data_center_config']['username']
    db_password = config['data_center_config']['password']
    db_name = config['data_center_config']['basename']
    return pymysql.connect(host=db_url,
                           port=int(db_port),
                           user=db_username,
                           password=db_password,
                           db=db_name,
                           charset='utf8mb4')

In [29]:
conn = getConn()

In [32]:
# 读取Binance
data_list = []
for year in range(2019, 2025):
    table_name = f'binance_spot_kline_daily_{year}'
    sql = f'''SELECT * FROM {table_name} WHERE symbol LIKE '%USDT';'''
    cs = conn.cursor()
    cs.execute(sql)
    res = cs.fetchall()
    data_list.append(pd.DataFrame(res))
    if year == 2024:
        columns_sql = '''SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS 
                         WHERE TABLE_NAME = %s ORDER BY ORDINAL_POSITION;'''
        cs.execute(columns_sql, (table_name,))
        columns = [col[0] for col in cs.fetchall()]
data = pd.concat(data_list, ignore_index=True)
data.columns = columns

In [34]:
data.to_csv('binance_data.csv', index=False)

# binance上第一天上线后币种 2020.01.01开始

In [3]:
binance_data = pd.read_csv('binance_data.csv')

In [4]:
binance_data = binance_data[['close_time_date', 'symbol']].drop_duplicates()
binance_data['close_time_date'] = binance_data['close_time_date'].apply(lambda x:x.split(' ')[0])
binance_data.sort_values(by='close_time_date', inplace=True)
binance_data.reset_index(inplace=True, drop=True)

In [67]:
tge_data = binance_data.groupby('symbol', as_index=False).agg(date=('close_time_date', 'first'))
tge_data['year'] = tge_data['date'].apply(lambda x:int(x.split('-')[0]))
tge_data = tge_data[tge_data['year']>2019].sort_values(by='date')
tge_data.reset_index(inplace=True, drop=True)
tge_data.rename(columns={'date':'tge_date'}, inplace=True)
tge_data['symbol'] = tge_data['symbol'].apply(lambda x:x[:-4])
tge_data.to_csv('tge_data.csv', index=False)

# Cmc获取市值数据

In [28]:

def getConnCmc():
    config = configparser.ConfigParser()
    config.read('config.ini', encoding='utf8')
    db_url = config['base_config']['host']
    db_port = config['base_config']['port']
    db_username = config['base_config']['username']
    db_password = config['base_config']['password']
    db_name = config['base_config']['basename']
    return pymysql.connect(host=db_url,
                           port=int(db_port),
                           user=db_username,
                           password=db_password,
                           db=db_name,
                           charset='utf8mb4')

In [29]:
conn = getConnCmc()

In [30]:
tge_data = pd.read_csv('tge_data.csv')
tge_data.head()

,symbol,tge_date,year
0,EUR,2020-01-04,2020
1,OGN,2020-01-10,2020
2,WRX,2020-02-06,2020
3,BNT,2020-02-07,2020
4,LSK,2020-02-07,2020


In [31]:
check_symbols = tge_data.groupby('year')['symbol'].unique().apply(list).to_dict()

In [74]:
# check_symbols[2020]

In [32]:
# 读取 cmc
mv_data_list = []
for year in range(2020, 2025):
    table_name = f'pv_allcoin_raw_data_daily_cmc_{year}'
    symbol_s = tuple(check_symbols[year]) 
    sql = f'''
            select * from {table_name} where symbol in {symbol_s} and market_cap > 0;
        '''
    cs = conn.cursor()
    cs.execute(sql)
    res = cs.fetchall()
    mv_data_list.append(pd.DataFrame(res))
    if year == 2024:
        columns_sql = '''SELECT COLUMN_NAME FROM INFORMATION_SCHEMA.COLUMNS 
                         WHERE TABLE_NAME = %s ORDER BY ORDINAL_POSITION;'''
        cs.execute(columns_sql, (table_name,))
        columns = [col[0] for col in cs.fetchall()]
mv_data = pd.concat(mv_data_list, ignore_index=True)
mv_data.columns = columns
mv_data.to_csv('cmc_data.csv', index=False)

# tge后的市值分析

In [2]:
mv_data = pd.read_csv('cmc_data.csv')
mv_data.drop_duplicates(subset=['symbol', 'date_time_ymd'], inplace=True)
mv_data.sort_values(by='date_time_ymd', inplace=True)
mv_data.reset_index(inplace=True, drop=True)
mv_data = mv_data[['symbol','date_time_ymd','market_cap','fully_diluted_market_cap']]

In [ ]:
tge_data_finall_list = []
for s in list(np.unique(mv_data['symbol'])):
    tge_date = mv_data[mv_data['symbol']==s].sort_values('date_time_ymd').reset_index(drop=True).loc[0,'date_time_ymd']
    tmp_data = mv_data[mv_data['symbol']==s].sort_values('date_time_ymd').reset_index(drop=True).iloc[1:4,:]
    if (tmp_data['market_cap'].isnull().sum()==3) | (tmp_data['fully_diluted_market_cap'].isnull().sum()==3):
        pass
    else:
        mv_mean = tmp_data['market_cap'].dropna().mean()
        fdv_mean = tmp_data['fully_diluted_market_cap'].dropna().mean()
    df = pd.DataFrame()
    df.loc[0,'tge_date'] = tge_date
    df.loc[0,'symbol'] = s
    df.loc[0,'mv_mean_3d_tge'] = mv_mean
    df.loc[0,'mv_fdv_3d_tge'] = fdv_mean
    tge_data_finall_list.append(df)
tge_data_finall = pd.concat(tge_data_finall_list, ignore_index=True)
tge_data_finall['year'] = tge_data_finall['tge_date'].apply(lambda x:int(x.split('-')[0]))
tge_data_finall.dropna(inplace=True)
tge_data_finall.sort_values(by='tge_date',inplace=True)
tge_data_finall.reset_index(inplace=True, drop=True)
tge_data_finall.to_csv('tge_data_finall.csv', index=False)

In [5]:
tge_data_finall = pd.read_csv('tge_data_finall.csv')
finall_analyze = pd.DataFrame(index=range(2020, 2025))
for y in range(2020, 2025):
    mv_mean_3d_tge = tge_data_finall[tge_data_finall['year']==y]['mv_mean_3d_tge'].median()
    mv_fdv_3d_tge = tge_data_finall[tge_data_finall['year']==y]['mv_fdv_3d_tge'].median()
    finall_analyze.loc[y, 'mv_mean_post_3d_tge'] = mv_mean_3d_tge
    finall_analyze.loc[y, 'fdv_mean_post_3d_tge'] = mv_fdv_3d_tge
finall_analyze

,mv_mean_post_3d_tge,fdv_mean_post_3d_tge
2020,1.431126e+08,6.161391e+08
2021,1.679616e+08,7.529309e+08
2022,2.215832e+08,6.222287e+08
2023,1.755526e+08,6.161391e+08
2024,2.596153e+08,9.434547e+08
